###  모델 학습 전 전처리

In [1]:
# GPU확인
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
# 메모리 확인
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Tue Dec  6 11:36:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    54W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### # 세팅

In [2]:
# pycaret 안될 때
!pip install -U tensorboard
!pip uninstall scikit-learn -y
!pip install -U pycaret scikit-learn
!pip install tensorflow==2.1.0
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow-gpu
!pip install keras==2.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 4.9 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.11.0 which is incompatible.
Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 4.5 MB/s 
     |████████████████████████████████| 31.2 MB 1.2 MB/s 
     |████████████████████████████████| 5.0 MB 76.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.1.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 377 kB 4.9 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
  Attempting uninstall: keras
    Found existing installation: ke

In [ ]:
import os, random, torch
import numpy as np
import pandas as pd
def setSeeds(seed = 9944):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)    
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
setSeeds()

from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pickle
from joblib import dump, load
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import preprocessing
import math

from pycaret.classification import *

plt.style.use('seaborn')
sns.set(font_scale = 2.5)
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
# import tensorflow_decision_forests as tfdf

In [ ]:
df_train_3d = pd.read_csv("../gdrive/MyDrive/EDA/dataset/body_3d_add.csv")
df_valid_3d = pd.read_csv("../gdrive/MyDrive/EDA/dataset/valid_body_3d_add.csv")

### # 데이터 전처리

In [ ]:
# string타입 : exercise, conditions
# frame, active는 train에 포함 X
exercise_list = ['플랭크', '푸시업', '니푸쉬업',  '굿모닝',  'Y - Exercise', '스탠딩 사이드 크런치',  '스탠딩 니업', '크런치', '바이시클 크런치' , '사이드 런지' ]
pts_list = ['pts.Nose.x', 'pts.Nose.y', 'pts.Nose.z',
       'pts.Left Eye.x', 'pts.Left Eye.y', 'pts.Left Eye.z', 'pts.Right Eye.x',
       'pts.Right Eye.y', 'pts.Right Eye.z', 'pts.Left Ear.x',
       'pts.Left Ear.y', 'pts.Left Ear.z', 'pts.Right Ear.x',
       'pts.Right Ear.y', 'pts.Right Ear.z', 'pts.Left Shoulder.x',
       'pts.Left Shoulder.y', 'pts.Left Shoulder.z', 'pts.Right Shoulder.x',
       'pts.Right Shoulder.y', 'pts.Right Shoulder.z', 'pts.Left Elbow.x',
       'pts.Left Elbow.y', 'pts.Left Elbow.z', 'pts.Right Elbow.x',
       'pts.Right Elbow.y', 'pts.Right Elbow.z', 'pts.Left Wrist.x',
       'pts.Left Wrist.y', 'pts.Left Wrist.z', 'pts.Right Wrist.x',
       'pts.Right Wrist.y', 'pts.Right Wrist.z', 'pts.Left Hip.x',
       'pts.Left Hip.y', 'pts.Left Hip.z', 'pts.Right Hip.x',
       'pts.Right Hip.y', 'pts.Right Hip.z', 'pts.Left Knee.x',
       'pts.Left Knee.y', 'pts.Left Knee.z', 'pts.Right Knee.x',
       'pts.Right Knee.y', 'pts.Right Knee.z', 'pts.Left Ankle.x',
       'pts.Left Ankle.y', 'pts.Left Ankle.z', 'pts.Right Ankle.x',
       'pts.Right Ankle.y', 'pts.Right Ankle.z', 'pts.Neck.x', 'pts.Neck.y',
       'pts.Neck.z', 'pts.Left Palm.x', 'pts.Left Palm.y', 'pts.Left Palm.z',
       'pts.Right Palm.x', 'pts.Right Palm.y', 'pts.Right Palm.z',
       'pts.Back.x', 'pts.Back.y', 'pts.Back.z', 'pts.Waist.x', 'pts.Waist.y',
       'pts.Waist.z', 'pts.Left Foot.x', 'pts.Left Foot.y', 'pts.Left Foot.z',
       'pts.Right Foot.x', 'pts.Right Foot.y', 'pts.Right Foot.z']
condition_list = [ 'condition0', 'condition1', 'condition2', 'condition3', 'condition4','exercise']
label_list = [  'value0', 'value1', 'value2', 'value3', 'value4']
angle_dict = {'spine' : ['pts.Neck', "pts.Back", "pts.Waist"], 
              "left_elbow":["pts.Left Wrist", "pts.Left Elbow", "pts.Left Shoulder"],
              "right_elbow":["pts.Right Wrist", "pts.Right Elbow", "pts.Right Shoulder"],
              "left_hand_position" : ["pts.Left Wrist", 'pts.Left Shoulder', "pts.Back"],
              "right_hand_position" : ["pts.Right Wrist", 'pts.Right Shoulder', "pts.Back"],
              "head" : ["pts.Nose", 'pts.Neck', "pts.Back"],
              "left_knee" : ["pts.Left Ankle", 'pts.Left Knee', "pts.Left Hip"],
              "right_knee" : ["pts.Right Ankle", 'pts.Right Knee', "pts.Right Hip"],
              "sight" : ["pts.Left Eye", 'pts.Neck', "pts.Left Shoulder"],
              "left_hand" : ["pts.Left Palm", "pts.Left Wrist", "pts.Left Elbow"],
              "right_hand" : ["pts.Right Palm", "pts.Right Wrist", "pts.Right Elbow"],
              "left_knee_elbow" : ["pts.Left Elbow", "pts.Waist", "pts.Left Knee"],
              "right_knee_elbow" : ["pts.Right Elbow", "pts.Waist", "pts.Right Knee"],
              "left_body_knee" : ["pts.Nose", "pts.Waist", "pts.Left Knee"],
              "right_body_knee" : ["pts.Nose", "pts.Waist", "pts.Right Knee"],
              "left_knee_hight" : ["pts.Left Knee", "pts.Left Hip", "pts.Back"],
              "right_knee_hight" : ["pts.Right Knee", "pts.Right Hip", "pts.Back"],
              "straight_beck" : ["pts.Hip", "pts.Back", "pts.Waist"],
              "left_leg_direction" : ["pts.Left Ankle", "pts.Left Knee", "pts.Waist"],
              "right_leg_direction" : ["pts.Right Ankle", "pts.Right Knee", "pts.Waist"],
              "arm_same_height" : ["pts.Left Elbow", "pts.Neck", "pts.Right Elbow"],
              "scapula" : ["pts.Left Shoulder", "pts.Back", "pts.Right Shoulder"],
              "left_knee_cross" : ["pts.Left Knee", "pts.Hip", "pts.Back"],
              "right_knee_cross" : ["pts.Right Knee", "pts.Hip", "pts.Right Elbow"],
              }
angle_list = list(angle_dict.keys())

In [ ]:
print(angle_list)

In [ ]:
def calculate_angle(a,b):
  tmp = (np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b)))
  if tmp > 1: tmp = 1
  if tmp < -1: tmp = -1
  theta = math.acos(tmp)
  return math.trunc(math.degrees(theta))

In [ ]:
# 각도 추가하기
def add_angle(df_train_3d, df_valid_3d):
  # 엉덩이 중앙 좌표 넣기
  for df in [df_train_3d, df_valid_3d]:
    df["pts.Hip.x"], df["pts.Hip.y"], df["pts.Hip.z"] = (df["pts.Left Hip.x"] + df["pts.Right Hip.x"])/2, (df["pts.Left Hip.y"] + df["pts.Right Hip.y"])/2, (df["pts.Left Hip.z"] + df["pts.Right Hip.z"])/2
  
  # 각도 계산 + 추가
  for key in angle_dict.keys():
    for df in [df_train_3d, df_valid_3d]:
      a,b = [],[]
      tmp = angle_dict[key]
      for i in [".x",".y",".z"]:
        a.append(df[tmp[0] + i] - df[tmp[1] + i])
        b.append(df[tmp[2] + i] - df[tmp[1] + i])
      a,b = np.array(a).T, np.array(b).T
      df[key] = list(map(calculate_angle, a,b))
  
  return df_train_3d, df_valid_3d

In [ ]:
# 운동 종목 추려내기 + null값, true, false값 바꿔주기, label encoding
def data_preprocessing(df_train_3d, df_valid_3d):
  # 사용하는 운동종목만 추려내기
  df_train_3d = df_train_3d[df_train_3d["exercise"].isin(exercise_list)]
  df_valid_3d = df_valid_3d[df_valid_3d["exercise"].isin(exercise_list)]

  # null값을 -1로 바꿔줌
  # True False값을 학습하기 쉽게 1과 0으로 바꿔줌
  df_train_3d = df_train_3d.replace("0", -1)
  df_valid_3d = df_valid_3d.replace("0", -1)
  df_train_3d = df_train_3d.replace(True, 1)
  df_train_3d = df_train_3d.replace(False, 0)
  df_valid_3d = df_valid_3d.replace(True, 1)
  df_valid_3d = df_valid_3d.replace(False, 0)
  df_train_3d = df_train_3d.replace("True", 1)
  df_train_3d = df_train_3d.replace("False", 0)
  df_valid_3d = df_valid_3d.replace("True", 1)
  df_valid_3d = df_valid_3d.replace("False", 0)

  # *   condition들의 값은 str임
  # *   예시로 "가슴의 충분한 이동"이 있음
  # *   str을 자연어 학습을 할 때처럼 숫자로 바꿔서 학습해도 되지만, 일단 지금은 각 condition에 해당하는 번호를 할당한 후 LabelEncoder로 값을 바꿔줌.
  # *   예시 => {"가슴의 충분한 이동" : 1}, {"견갑골 하강 유지" : 2} 
  le = preprocessing.LabelEncoder()
  labels = []
  labels.extend(exercise_list)
  for i in df_train_3d.columns:
    if "condition" in i:
      labels.extend(df_train_3d[df_train_3d[i] != -1][i].unique())
      labels.extend(df_valid_3d[df_valid_3d[i]!= -1][i].unique())
  le.fit(labels)
  for i in le.classes_:
    df_train_3d = df_train_3d.replace(i, le.transform([i])[0])
    df_valid_3d = df_valid_3d.replace(i, le.transform([i])[0])

  return df_train_3d, df_valid_3d, le

In [ ]:
# df_train_3d, df_valid_3d = add_angle(df_train_3d, df_valid_3d)
# df_train_3d.to_csv("../gdrive/MyDrive/EDA/tabular_data/unprocessed_body_3d.csv")
# df_valid_3d.to_csv("../gdrive/MyDrive/EDA/tabular_data/unprocessed_valid_body_3d.csv")
# df_train_3d, df_valid_3d,le = data_preprocessing(df_train_3d, df_valid_3d)
df_train_3d = pd.read_csv("../gdrive/MyDrive/EDA/tabular_data/unprocessed_body_3d.csv")
df_valid_3d = pd.read_csv("../gdrive/MyDrive/EDA/tabular_data/unprocessed_valid_body_3d.csv")
df_train_3d, df_valid_3d,le = data_preprocessing(df_train_3d, df_valid_3d)

In [ ]:
def matrix(y_valid, prediction, label_list, model):
  total = 0
  if model == "lstm":
    y_valid = np.array(y_valid).T
  if model == "random_forest": #  or model == 'lstm':
    prediction = np.array(prediction).T
  for idx, tmp in enumerate(label_list):
    if model == "lstm":
      acc = accuracy_score(y_valid[idx], prediction[idx])
    elif model == "pycaret":
      acc = accuracy_score(y_valid[tmp], prediction[idx]["Label"])
    else:
      acc = accuracy_score(y_valid[tmp], prediction[idx])

    print(f"{tmp}의 accuracy : {acc}")
    total += acc
  print(f"총 정확도 :{total/5}")

# # **LSTM**

##### 1) LSTM 데이터셋 만들기

In [ ]:
# https://data-analysis-expertise.tistory.com/67
# 참고한 블로그

In [ ]:
def make_train_test(time_steps, train_list, label_list):
  """
  input:
   time_steps: sequence data를 얼마나 볼건지
  # LSTM과 GRU는 3차원 배열의 입려값을 사용한다. 
  # 데이터를 3차원으로 재구성 한다. 이때 3차원 입력 데이터는 1. samples(총 데이터 크기) 2. time steps 3. features(변수 개수) 이렇게 3개의 요소로 구성된다.
  output:  
    x_train, y_train : 학습을 위한 데이터와 레이블
    x_valid, y_valid : inference를 위한 데이터와 레이블
  """

  train_3d = df_train_3d[train_list + label_list + ["frame"]]
  valid_3d = df_valid_3d[train_list + label_list + ["frame"]]

  # LSTM은 sequence 데이터를 확인할 수 있다. time steps로 원본 데이터 슬라이싱하는 과정
  # 학습시키는 데이터는 time_steps만큼 슬라이싱함. 레이블은 현재 운동 종목과 타입, 상태를 넣어줌. 즉 슬라이싱 안함.
  x_train, y_train, x_valid, y_valid = [],[],[],[]
  for i in range(time_steps, len(train_3d)+1):
    if train_3d.iloc[i-time_steps]["frame"] == train_3d.iloc[i-1]["frame"]:
      x_train.append(train_3d[i-time_steps:i][train_list])
      y_train.append(train_3d.iloc[i-1][label_list])

  x_train, y_train = np.array(x_train), np.array(y_train)

  # 3차원 입력으로 재구성
  # np.reshape(samples, time_steps, features)로 만든다. 
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], len(train_list)))

  for i in range(time_steps, len(valid_3d)+1):
    if valid_3d.iloc[i-time_steps]["frame"] == valid_3d.iloc[i-1]["frame"]:
      x_valid.append(valid_3d[i-time_steps:i][train_list])
      y_valid.append(valid_3d.iloc[i-1][label_list])
  x_valid, y_valid = np.array(x_valid), np.array(y_valid)

  x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], len(train_list)))
  
  return x_train, y_train, x_valid, y_valid


In [ ]:
x_train, y_train, x_valid, y_valid = make_train_test(5, pts_list + condition_list, label_list ) # 5는 time series 길이

In [ ]:
x_angle_train, y_angle_train, x_angle_valid, y_angle_valid = make_train_test(5, angle_list + condition_list, label_list ) # 5는 time series 길이

##### 2) LSTM Train & inference

In [ ]:
# https://stackoverflow.com/questions/68844792/lstm-will-not-use-cudnn-kernels-since-it-doesnt-meet-the-criteria-it-will-use # activation == tanh이 되어야함. 
# specifically accelerate LSTM and GRU layers. These GRU/LSTM layers can only be accelerated if they meet a certain criteria
# activation == tanh
def LSTM_multiclass_multioutput(x_train, y_train, x_valid, y_valid, le, epoch = 20, batch_size = 150, verbose = 2, optimizer = 'adam', loss = 'categorical_crossentropy' , activation = 'tanh'):
  from keras.models import Sequential, Model
  from keras.layers import Dense, SimpleRNN, GRU, LSTM, Embedding, Input
  from tensorflow.keras.optimizers import SGD, Adam
  from keras.utils import to_categorical
  from keras import Input
  starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
  repetitions = 300
  timings=np.zeros((repetitions,1))

  # 데이터 형태 바꿔주기
  for i in [x_train, y_train, x_valid, y_valid]:
    i = np.asarray(i).astype(np.float32)
   # one hot encode output variable
  
  x = Input(shape = (x_train.shape[1], x_train.shape[2]))
  x_lstm1 = LSTM(128, activation =activation, name = 'lstm1', return_sequences=True)(x)
  x_lstm2 = LSTM(128, activation = activation, name = 'lstm2')(x_lstm1)
  # binary classification에서 dense예측을 1로 하고 argmax를 한게 잘못이었음. 그러니까 하나의 값으로 예측하지,,
  out1 = Dense(1, activation='sigmoid', name = 'out1')(x_lstm2)
  out2 = Dense(1, activation='sigmoid', name = 'out2')(x_lstm2)
  out3 = Dense(1, activation='sigmoid', name = 'out3')(x_lstm2)
  out4 = Dense(3, activation='softmax', name = 'out4')(x_lstm2)
  out5 = Dense(3, activation='softmax', name = 'out5')(x_lstm2)
  model = Model(inputs = x, outputs=[out1, out2, out3, out4, out5])
  print(model.summary())
  model.compile(optimizer=Adam(lr=1e-4), loss = {'out1' : 'binary_crossentropy','out2' : 'binary_crossentropy','out3' : 'binary_crossentropy',
                                             'out4' :'categorical_crossentropy','out5' :'categorical_crossentropy'}) # , loss_weights = {})

  # fitting
  model.fit(x_train, 
            [y_train[:,0], y_train[:,1], y_train[:,2], to_categorical(y_train[:,3], num_classes = 3), to_categorical(y_train[:,4], num_classes = 3)], 
            epochs = epoch, 
            batch_size = batch_size, 
            verbose = verbose)
            # validation_data = (x_valid, y_valid),
            # callbacks=[tensorboard_callback])

  ### inference time ###
  starter.record()
  tmp = model.predict(x_valid[:2])
  ender.record()
  torch.cuda.synchronize()
  curr_time = starter.elapsed_time(ender)
  print("*"*10)
  print(f"inference time : {curr_time}")
  print("*"*10)
  ### inference time ###

  prediction = model.predict(x_valid)
  pred = []
  for i in range(0,3):
    pred.append(list(map(np.round, prediction[i])))
    print(pred)
  for i in range(3,5):
   pred.append(np.argmax(prediction[i], axis=-1))
  for i in range(3,5):
    for j in range(len(pred[i])):
      if pred[i][j] == 2:
        pred[i][j] = -1
  return model, pred

In [ ]:
lstm,prediction = LSTM_multiclass_multioutput(x_train, y_train, x_valid, y_valid, le, epoch = 20, batch_size = 150, verbose = 2, optimizer = 'adam', loss = 'mean_squared_error' , activation = 'tanh' )
lstm_angle,prediction_angle = LSTM_multiclass_multioutput(x_angle_train, y_angle_train, x_angle_valid, y_angle_valid, le, epoch = 20, batch_size = 150, verbose = 2, optimizer = 'adam', loss = 'mean_squared_error' , activation = 'tanh' )

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 5, 78)        0                                            
__________________________________________________________________________________________________
lstm1 (LSTM)                    (None, 5, 128)       105984      input_7[0][0]                    
__________________________________________________________________________________________________
lstm2 (LSTM)                    (None, 128)          131584      lstm1[0][0]                      
__________________________________________________________________________________________________
out1 (Dense)                    (None, 1)            129         lstm2[0][0]                      
____________________________________________________________________________________________

KeyboardInterrupt: ignored

In [ ]:

def LSTM(x_train, y_train, x_valid, y_valid, le, epoch = 20, batch_size = 150, verbose = 2, optimizer = 'adam', loss = 'categorical_crossentropy' , activation = 'relu'):
  from keras.models import Sequential
  from keras.layers import Dense, SimpleRNN, GRU, LSTM
  from tensorflow.keras.optimizers import SGD
  from keras.utils import to_categorical


  starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
  repetitions = 300
  timings=np.zeros((repetitions,1))

  # architecture
  LSTM_model = Sequential()
  LSTM_model.add(LSTM(
      units = 64, 
      return_sequences = True,
      input_shape = (x_train.shape[1], x_train.shape[2]),
      activation = activation # relu
  ))
  #   keras.layers.Dropout(0.2),
  LSTM_model.add(LSTM(units = 64, activation = activation))
  LSTM_model.add(Dense(units=len(label_list), activation='softmax'))

  # compiling
  # LSTM_model.compile(optimizer = SGD(lr = 0.01, decay = 1e-7, momentum = 0.9, nesterov = False), 
  #                   loss = "mean_squared_error") or optimizer = 'adam'
  LSTM_model.compile(optimizer=optimizer,
                    loss=loss,
                    metrics=['accuracy'])
          #    loss='categorical_crossentropy',)

  
  # 데이터 형태 바꿔주기
  for i in [x_train, y_train, x_valid, y_valid]:
    i = np.asarray(i).astype(np.float32)

  # logs
  # log_dir = "../gdrive/MyDrive/EDA/dataset/logs/lstm"
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

  # fitting
  LSTM_model.fit(x_train, 
                 y_train, 
                 epochs = epoch, 
                 batch_size = batch_size, 
                 verbose = verbose)
                 # validation_data = (x_valid, y_valid),
                 # callbacks=[tensorboard_callback])

  ### inference time ###
  starter.record()
  prediction = LSTM_model.predict(x_valid[:2])
  ender.record()
  torch.cuda.synchronize()
  curr_time = starter.elapsed_time(ender)
  print("*"*10)
  print(f"inference time : {curr_time}")
  print("*"*10)
  ### inference time ###


  prediction = LSTM_model.predict(x_valid)
  prediction = [[round(j) for j in prediction[i]] for i in range(prediction.shape[0]) ]

  return LSTM_model, prediction

In [ ]:
lstm,prediction = LSTM(x_train, y_train, x_valid, y_valid, le, epoch = 1, batch_size = 150, verbose = 2, optimizer = 'adam', loss = 'mean_squared_error' , activation = 'tanh' )
lstm_angle,prediction_angle = LSTM(x_angle_train, y_angle_train, x_angle_valid, y_angle_valid, le, epoch = 1, batch_size = 150, verbose = 2, optimizer = 'adam', loss = 'mean_squared_error' , activation = 'tanh' )

Epoch 1/1
 - 8s - loss: 0.4203 - accuracy: 0.3374
**********
inference time : 148.0069122314453
**********
Epoch 1/1
 - 7s - loss: 0.4208 - accuracy: 0.3401
**********
inference time : 154.46630859375
**********


In [ ]:
matrix(y_valid, prediction, label_list, "lstm")
matrix(y_angle_valid, prediction_angle, label_list, "lstm")

In [ ]:
for i in range(5):
  print(set(prediction[i]))

# # **Random Forest**
*   scikit-learn만 randomforest모델로 multi-output task를 지원해주기때문에 scikit-learn사용

In [ ]:
df_valid_3d[angle_list].iloc[:2]

,spine,left_elbow,right_elbow,left_hand_position,right_hand_position,head,left_knee,right_knee,sight,left_hand,...,right_body_knee,left_knee_hight,right_knee_hight,straight_beck,left_leg_direction,right_leg_direction,arm_same_height,scapula,left_knee_cross,right_knee_cross
256,173,44,109,69,19,98,77,108,53,91,...,143,77,147,22,80,119,38,119,82,129
257,107,36,177,34,15,74,107,100,123,30,...,139,91,130,20,138,113,25,149,95,108


In [ ]:
def random_forest(status, train_list, label_list, name = ""):
  if status == "train":
    # training code
    forest = RandomForestClassifier(random_state = 1, bootstrap=True)
    multi_target_forest = MultiOutputClassifier(forest, n_jobs=None)
    multi_target_forest = multi_target_forest.fit(df_train_3d.loc[:, train_list], df_train_3d.loc[:, label_list])
    dump(multi_target_forest, f'../gdrive/MyDrive/EDA/dataset/multi_target_forest{name}.pkl')
  elif status == "valid":

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = 300
    timings=np.zeros((repetitions,1))

    ### inference time ###
    starter.record()
    model = load( f'../gdrive/MyDrive/EDA/dataset/multi_target_forest{name}.pkl')
    prediction = model.predict(df_valid_3d[train_list].iloc[:2])
    ender.record()
    torch.cuda.synchronize()
    curr_time = starter.elapsed_time(ender)
    print("*"*10)
    print(f"inference time : {curr_time}")
    print("*"*10)
    ### inference time ###
    
    prediction = model.predict(df_valid_3d.loc[:, train_list])
    matrix(df_valid_3d, prediction, label_list, "random_forest")

In [ ]:
# random_forest("train", pts_list + condition_list, label_list)
random_forest("valid", pts_list + condition_list, label_list)

**********
inference time : 1204.0140380859375
**********
value0의 accuracy : 0.5547733516483516
value1의 accuracy : 0.514165521978022
value2의 accuracy : 0.49905563186813184
value3의 accuracy : 0.698489010989011
value4의 accuracy : 0.7667410714285714
총 정확도 :0.6066449175824176


In [ ]:
# random_forest("train", angle_list + condition_list, label_list, name = "_angle")
random_forest("valid", angle_list + condition_list, label_list, name = "_angle")

**********
inference time : 1452.7723388671875
**********
value0의 accuracy : 0.5554601648351648
value1의 accuracy : 0.5723729395604396
value2의 accuracy : 0.5353708791208791
value3의 accuracy : 0.7300824175824175
value4의 accuracy : 0.7657967032967034
총 정확도 :0.6318166208791209


# # Pycaret

In [ ]:
# https://velog.io/@ezoo0422/Python-pycaret%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EB%AA%A8%EB%8D%B8-%EC%84%A0%EC%A0%95%ED%95%98%EA%B8%B0#:~:text=pycaret%EC%9D%84%20%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0%20%EC%A0%84%EC%97%90,%EC%B9%98%EB%A9%B4%20data%EA%B0%80%20%EC%84%A4%EC%A0%95%EB%90%A9%EB%8B%88%EB%8B%A4.
# 참고
from pycaret.classification import *

In [ ]:
def pycaret_blending(status, train_list, name = ""):
  if status == "train":
    for i in range(5):
      model = setup(data = df_train_3d[train_list + [f'condition{i}', f'value{i}']], target = f'value{i}', session_id=9944)
      best_model_3 = compare_models(n_select = 3)
      blender_3 = blend_models(best_model_3)
      final_model = finalize_model(blender_3)
      save_model(final_model, f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret{name}')
    return 0
  elif status == "valid":
    prediction = []
    for i in range(5):
      model = load( f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret{name}.pkl')
      pred = predict_model(model, data = df_valid_3d[train_list + [f'condition{i}']])
      prediction.append(pred)
    return prediction

In [ ]:
def pycaret_bestmodel(status, train_list, name = ""):
  if status == "train":
    for i in range(5):
      model = setup(data = df_train_3d[train_list + [f'condition{i}', f'value{i}']], target = f'value{i}', session_id=9944)
      best_model = compare_models(n_select = 1)
      save_model(best_model, f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret_bestmodel{name}')
    return 0
  elif status == "valid":
    prediction = []
    for i in range(5):
      model = load( f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret_bestmodel{name}.pkl')
      pred = predict_model(model, data = df_valid_3d[train_list + [f'condition{i}']])
      prediction.append(pred)
    return prediction

In [ ]:
df_valid_3d[ pts_list + ["exercise"]].iloc[1]

pts.Nose.x          -47.337494
pts.Nose.y          130.494019
pts.Nose.z          -23.802925
pts.Left Eye.x      -50.873493
pts.Left Eye.y      130.988174
                       ...    
pts.Left Foot.z     -55.584106
pts.Right Foot.x     46.150242
pts.Right Foot.y    106.190994
pts.Right Foot.z    -44.487041
exercise             36.000000
Name: 257, Length: 73, dtype: float64

In [ ]:
def pycaret_lgbm(status, train_list, name = ""):
  if status == "train":
    for i in range(5):
      model = setup(data = df_train_3d[train_list + [f'condition{i}', f'value{i}']], target = f'value{i}', session_id=9944)
      lgbm = create_model('lightgbm')
      tuned_lgbm = tune_model(lgbm, n_iter = 20, optimize = "Accuracy")
      save_model(tuned_lgbm, f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret_lgbm{name}')
    return 0
  elif status == "valid":
    prediction = []

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = 300
    timings=np.zeros((repetitions,1))
    ####inference time######
    starter.record()
    for i in range(5):
      model = load( f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret_lgbm{name}.pkl')
      tmp = df_valid_3d[train_list + [f'condition{i}']].iloc[:1]
      tmp = predict_model(model, data = tmp)
      print("prediction")
    ender.record()
    torch.cuda.synchronize()
    curr_time = starter.elapsed_time(ender)
    print(f"inference time : {curr_time}")
    ####inference time######
    for i in range(5):
      model = load( f'../gdrive/MyDrive/EDA/dataset/value{i}_pycaret_lgbm{name}.pkl')
      pred = predict_model(model, data = df_valid_3d[train_list + [f'condition{i}']])
      prediction.append(pred)
    return prediction

In [ ]:
# pycaret_lgbm("train", pts_list + ["exercise"], name = "_pts")
prediction = pycaret_lgbm("valid", pts_list + ["exercise"], name = '_pts')
matrix(df_valid_3d, prediction, label_list, "pycaret")

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value0',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

prediction


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value2',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

prediction


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value3',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

...

prediction


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value0',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

prediction
inference time : 1675.4432373046875


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value1',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

value0의 accuracy : 0.5497081043956044
value1의 accuracy : 0.5119333791208791
value2의 accuracy : 0.49373282967032966
value3의 accuracy : 0.6480082417582418
value4의 accuracy : 0.7529189560439561
총 정확도 :0.5912603021978022


In [ ]:
pycaret_lgbm("train", angle_list + ["exercise"], name = "_angle")
prediction = pycaret_lgbm("valid", angle_list + ["exercise"], name = "_angle")
matrix(df_valid_3d, prediction, label_list, "pycaret")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8604,0.9605,0.8569,0.8601,0.8594,0.7883,0.7891
1,0.8607,0.9574,0.8574,0.8603,0.8598,0.7887,0.7895
2,0.8600,0.9579,0.8580,0.8595,0.8595,0.7880,0.7883
3,0.8616,0.9579,0.8583,0.8612,0.8607,0.7901,0.7908
4,0.8637,0.9602,0.8595,0.8637,0.8625,0.7930,0.7943
5,0.8604,0.9567,0.8574,0.8600,0.8596,0.7884,0.7890
6,0.8630,0.9578,0.8593,0.8628,0.8620,0.7922,0.7931
7,0.8669,0.9606,0.8632,0.8668,0.8659,0.7981,0.7990
8,0.8609,0.9565,0.8574,0.8606,0.8599,0.7890,0.7899


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.2, max_depth=-1,
               min_child_samples=66, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=270, n_jobs=-1, num_leaves=150, objective=None,
               random_state=9944, reg_alpha=0.05, reg_lambda=0.005,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
INFO:logs:tune_model() succesfully completed......................................
INFO:logs:Initializing save_model()
INFO:logs:save_model(model=LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='s

Transformation Pipeline and Model Successfully Saved


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value0',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

prediction


...

prediction


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value0',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

prediction
inference time : 1460.6806640625


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='value1',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_sample

value0의 accuracy : 0.5409512362637363
value1의 accuracy : 0.5562328296703297
value2의 accuracy : 0.5115899725274725
value3의 accuracy : 0.7071600274725275
value4의 accuracy : 0.7584993131868132
총 정확도 :0.6148866758241759


In [ ]:
pycaret_func("train", angle_list + ["exercise"], name = "_angle")
prediction = pycaret_func("valid", angle_list + ["exercise"], name = "_angle")
matrix(df_valid_3d, prediction, label_list, "pycaret")

# # Saint

In [ ]:
!pip install pytorch_tabnet

In [ ]:
import torch
import torch.optim
import torch.nn as nn
import torch.nn.functional as F 
import copy
import re
import os
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from pycaret.classification import *
import random
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
args = {'transformer_depth':1,
        'attention_heads':4, # less than 4
        'attention_dropout':0.8,'ff_dropout':0.0,  
        'embedding_size':32 # less than 32
        'attentiontype':'colrow', # ['col','colrow','row','justmlp','attn','attnmlp']
        'optimizer':'AdamW', # ['AdamW','Adam','SGD']
        'scheduler':'cosine', # ['cosine','linear']
        'lr' : 0.0001,
        'epochs':100,
        'batchsize' : 128,
        'savemodelroot':'../gdrive/MyDrive/bestmodels'
}
criterion = nn.CrossEntropyLoss()
# if arge.optimizer == 'SGD':
#     optimizer = optim.SGD(model.parameters(), lr=args.lr,
#                           momentum=0.9, weight_decay=5e-4)
#     from utils import get_scheduler
#     scheduler = get_scheduler(opt, optimizer)
# elif opt.optimizer == 'Adam':
#     optimizer = optim.Adam(model.parameters(),lr=opt.lr)
# elif opt.optimizer == 'AdamW':
#     optimizer = optim.AdamW(model.parameters(),lr=opt.lr)

In [ ]:


if opt.optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=opt.lr,
                          momentum=0.9, weight_decay=5e-4)
    from utils import get_scheduler
    scheduler = get_scheduler(opt, optimizer)
elif opt.optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(),lr=opt.lr)
elif opt.optimizer == 'AdamW':
    optimizer = optim.AdamW(model.parameters(),lr=opt.lr)
best_valid_auroc = 0
best_valid_accuracy = 0
best_test_auroc = 0
best_test_accuracy = 0
best_valid_rmse = 100000
print('Training begins now.')
for epoch in range(opt.epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        # x_categ is the the categorical data, x_cont has continuous data, y_gts has ground truth ys. cat_mask is an array of ones same shape as x_categ and an additional column(corresponding to CLS token) set to 0s. con_mask is an array of ones same shape as x_cont. 
        x_categ, x_cont, y_gts, cat_mask, con_mask = data[0].to(device), data[1].to(device),data[2].to(device),data[3].to(device),data[4].to(device)

        # We are converting the data to embeddings in the next step
        _ , x_categ_enc, x_cont_enc = embed_data_mask(x_categ, x_cont, cat_mask, con_mask,model,vision_dset)           
        reps = model.transformer(x_categ_enc, x_cont_enc)
        # select only the representations corresponding to CLS token and apply mlp on it in the next step to get the predictions.
        y_reps = reps[:,0,:]
        
        y_outs = model.mlpfory(y_reps)
        if opt.task == 'regression':
            loss = criterion(y_outs,y_gts) 
        else:
            loss = criterion(y_outs,y_gts.squeeze()) 
        loss.backward()
        optimizer.step()
        if opt.optimizer == 'SGD':
            scheduler.step()
        running_loss += loss.item()
    # print(running_loss)
    if opt.active_log:
        wandb.log({'epoch': epoch ,'train_epoch_loss': running_loss, 
        'loss': loss.item()
        })
    if epoch%5==0:
            model.eval()
            with torch.no_grad():
                if opt.task in ['binary','multiclass']:
                    accuracy, auroc = classification_scores(model, validloader, device, opt.task,vision_dset)
                    test_accuracy, test_auroc = classification_scores(model, testloader, device, opt.task,vision_dset)

                    print('[EPOCH %d] VALID ACCURACY: %.3f, VALID AUROC: %.3f' %
                        (epoch + 1, accuracy,auroc ))
                    print('[EPOCH %d] TEST ACCURACY: %.3f, TEST AUROC: %.3f' %
                        (epoch + 1, test_accuracy,test_auroc ))
                    if opt.active_log:
                        wandb.log({'valid_accuracy': accuracy ,'valid_auroc': auroc })     
                        wandb.log({'test_accuracy': test_accuracy ,'test_auroc': test_auroc })  
                    if opt.task =='multiclass':
                        if accuracy > best_valid_accuracy:
                            best_valid_accuracy = accuracy
                            best_test_auroc = test_auroc
                            best_test_accuracy = test_accuracy
                            torch.save(model.state_dict(),'%s/bestmodel.pth' % (modelsave_path))
                    else:
                        if accuracy > best_valid_accuracy:
                            best_valid_accuracy = accuracy
                        # if auroc > best_valid_auroc:
                        #     best_valid_auroc = auroc
                            best_test_auroc = test_auroc
                            best_test_accuracy = test_accuracy               
                            torch.save(model.state_dict(),'%s/bestmodel.pth' % (modelsave_path))

                else:
                    valid_rmse = mean_sq_error(model, validloader, device,vision_dset)    
                    test_rmse = mean_sq_error(model, testloader, device,vision_dset)  
                    print('[EPOCH %d] VALID RMSE: %.3f' %
                        (epoch + 1, valid_rmse ))
                    print('[EPOCH %d] TEST RMSE: %.3f' %
                        (epoch + 1, test_rmse ))
                    if opt.active_log:
                        wandb.log({'valid_rmse': valid_rmse ,'test_rmse': test_rmse })     
                    if valid_rmse < best_valid_rmse:
                        best_valid_rmse = valid_rmse
                        best_test_rmse = test_rmse
                        torch.save(model.state_dict(),'%s/bestmodel.pth' % (modelsave_path))
            model.train()
              

In [ ]:
class Saint(nn.Module):
    
    def __init__(self, args):
        super(Saint, self).__init__()
        self.args = args
        self.device = args.device

        self.hidden_dim = self.args.hidden_dim
        # self.dropout = self.args.dropout
        self.dropout = 0.
        
        ### Embedding 
        # ENCODER embedding
        # test, question, tag
        self.embedding_features = nn.ModuleList([])
        for value in self.args.n_embedding_layers:
            self.embedding_features.append(nn.Embedding(value + 1, self.hidden_dim // self.args.dim_div))
        
        # encoder combination projection
        self.enc_comb_proj = nn.Linear((self.hidden_dim//self.args.dim_div)*len(self.args.n_embedding_layers), self.hidden_dim)

        # DECODER embedding
        # interaction은 현재 correct으로 구성되어있다. correct(1, 2) + padding(0)
        self.embedding_interaction = nn.Embedding(3, self.hidden_dim//self.args.dim_div)
        
        # decoder combination projection
        self.dec_comb_proj = nn.Linear((self.hidden_dim//self.args.dim_div)*(len(self.args.n_embedding_layers)+1), self.hidden_dim)

        # Positional encoding
        self.pos_encoder = PositionalEncoding(self.hidden_dim, self.dropout, self.args.max_seq_len)
        self.pos_decoder = PositionalEncoding(self.hidden_dim, self.dropout, self.args.max_seq_len)
        

        self.transformer = nn.Transformer(
            d_model=self.hidden_dim, 
            nhead=self.args.n_heads,
            num_encoder_layers=self.args.n_layers, 
            num_decoder_layers=self.args.n_layers, 
            dim_feedforward=self.hidden_dim, 
            dropout=self.dropout, 
            activation='relu')

        self.fc = nn.Linear(self.hidden_dim, 1)
        self.activation = nn.Sigmoid()

        self.enc_mask = None
        self.dec_mask = None
        self.enc_dec_mask = None
    
    def get_mask(self, seq_len):
        mask = torch.from_numpy(np.triu(np.ones((seq_len, seq_len)), k=1))

        return mask.masked_fill(mask==1, float('-inf'))

    def forward(self, input):
        _, mask, interaction, index = input[-4:]

        batch_size = interaction.size(0)
        seq_len = interaction.size(1)

        # 신나는 embedding
        # ENCODER
        embed_features = []
        for _input, _embedding_feature in zip(input[:-4], self.embedding_features):
            value = _embedding_feature(_input)
            embed_features.append(value)

        embed_enc = torch.cat(embed_features, 2)

        embed_enc = self.enc_comb_proj(embed_enc)
        
        # DECODER     
        embed_features = []
        for _input, _embedding_feature in zip(input[:-4], self.embedding_features):
            value = _embedding_feature(_input)
            embed_features.append(value)

        embed_interaction = self.embedding_interaction(interaction)
        embed_features = [embed_interaction] + embed_features
        embed_dec = torch.cat(embed_features, 2)

        embed_dec = self.dec_comb_proj(embed_dec)

        # ATTENTION MASK 생성
        # encoder하고 decoder의 mask는 가로 세로 길이가 모두 동일하여
        # 사실 이렇게 3개로 나눌 필요가 없다
        if self.enc_mask is None or self.enc_mask.size(0) != seq_len:
            self.enc_mask = self.get_mask(seq_len).to(self.device)
            
        if self.dec_mask is None or self.dec_mask.size(0) != seq_len:
            self.dec_mask = self.get_mask(seq_len).to(self.device)
            
        if self.enc_dec_mask is None or self.enc_dec_mask.size(0) != seq_len:
            self.enc_dec_mask = self.get_mask(seq_len).to(self.device)
            
  
        embed_enc = embed_enc.permute(1, 0, 2)
        embed_dec = embed_dec.permute(1, 0, 2)
        
        # Positional encoding
        embed_enc = self.pos_encoder(embed_enc)
        embed_dec = self.pos_decoder(embed_dec)
        
        out = self.transformer(embed_enc, embed_dec,
                               src_mask=self.enc_mask,
                               tgt_mask=self.dec_mask,
                               memory_mask=self.enc_dec_mask)

        out = out.permute(1, 0, 2)
        out = out.contiguous().view(batch_size, -1, self.hidden_dim)
        out = self.fc(out)

        preds = self.activation(out).view(batch_size, -1)

        return preds


In [ ]:
def run(args, train_data, valid_data, test_data):
    if args.use_pseudo:
        pseudo_labels = pd.read_csv(args.pseudo_label_file) # '/opt/ml/p4-dkt-no_caffeine_no_gain/highest.csv'
        pseudo_labels = pseudo_labels['prediction'].to_numpy()
        pseudo_labels = np.where(pseudo_labels >= 0.5, 1, 0)

        pseudo_train_data = update_train_data(pseudo_labels, train_data, test_data)
        train_data = pseudo_train_data

    print(f"# of train data : {len(train_data)}")
    print(f"# of valid data : {len(valid_data)}")
    print()
    train_loader, valid_loader = get_loaders(args, train_data, valid_data)
    
    # only when using warmup scheduler
    args.total_steps = int(len(train_loader.dataset) / args.batch_size) * (args.n_epochs)
    args.warmup_steps = args.total_steps // 10

    print(args)
    model_dir = os.path.join(args.model_dir, args.model_name)
    os.makedirs(model_dir, exist_ok=True)
    json.dump(
        vars(args),
        open(f"{model_dir}/exp_config.json", "w"),
        indent=2,
        ensure_ascii=False,
    )
    
    print(f"\n{model_dir}/exp_config.json is saved!\n")
            
    model = get_model(args)
    if args.use_finetune:
        load_state = torch.load(args.trained_model)
        model.load_state_dict(load_state['state_dict'], strict=True)
        print(f"{args.trained_model} is loaded!")

    optimizer = get_optimizer(model, args)
    scheduler = get_scheduler(optimizer, args)

    best_auc = -1
    early_stopping_counter = 0
    for epoch in range(args.n_epochs):

        print(f"Start Training: Epoch {epoch}")
        
        ### TRAIN
        train_auc, train_acc, train_loss = train(train_loader, model, optimizer, args)
        
        ### VALID
        auc, acc, val_loss = validate(valid_loader, model, args)

        ### TODO: model save or early stopping
        if args.use_wandb:
            wandb.log({"train_loss": train_loss, "train_auc": train_auc, "train_acc": train_acc,
                       "val_loss": val_loss, "valid_auc": auc, "valid_acc": acc})

        if auc > best_auc:
            best_auc = auc
            # torch.nn.DataParallel로 감싸진 경우 원래의 model을 가져옵니다.
            model_to_save = model.module if hasattr(model, 'module') else model
            model_name = 'model_epoch' + str(epoch) + ".pt"
            save_checkpoint({
                'epoch': epoch,
                'state_dict': model_to_save.state_dict(),
                },
                 model_dir, model_name,
            )
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= args.patience:
                print(f'EarlyStopping counter: {early_stopping_counter} out of {args.patience}')
                break

        # scheduler
        if args.scheduler == 'plateau':
            scheduler.step(best_auc)
        else:
            scheduler.step()


def train(train_loader, model, optimizer, args):
    model.train()

    total_preds = []
    total_targets = []
    losses = []
    for step, batch in enumerate(train_loader):
        input = process_batch(batch, args)
        preds = model(input)

        targets = input[-4] # correct

        loss = compute_loss(preds, targets)
        update_params(loss, model, optimizer, args)

        if step % args.log_steps == 0:
            print(f"Training steps: {step} Loss: {str(loss.item())}")
        
        # predictions
        preds = preds[:,-1]
        targets = targets[:,-1]

        if args.device == 'cuda':
            preds = preds.to('cpu').detach().numpy()
            targets = targets.to('cpu').detach().numpy()
        else: # cpu
            preds = preds.detach().numpy()
            targets = targets.detach().numpy()
        
        total_preds.append(preds)
        total_targets.append(targets)
        losses.append(loss)
      

    total_preds = np.concatenate(total_preds)
    total_targets = np.concatenate(total_targets)

    # Train AUC / ACC
    auc, acc = get_metric(total_targets, total_preds)
    loss_avg = sum(losses)/len(losses)
    print(f'TRAIN AUC : {auc} ACC : {acc}')
    return auc, acc, loss_avg
    

def validate(valid_loader, model, args):
    model.eval()

    total_preds = []
    total_targets = []
    losses = []
    for step, batch in enumerate(valid_loader):
        input = process_batch(batch, args)

        preds = model(input)

        targets = input[-4] # correct

        loss = compute_loss(preds, targets)

        # predictions
        preds = preds[:,-1]
        targets = targets[:,-1]
    
        if args.device == 'cuda':
            preds = preds.to('cpu').detach().numpy()
            targets = targets.to('cpu').detach().numpy()
        else: # cpu
            preds = preds.detach().numpy()
            targets = targets.detach().numpy()

        total_preds.append(preds)
        total_targets.append(targets)
        losses.append(loss)

    total_preds = np.concatenate(total_preds)
    total_targets = np.concatenate(total_targets)

    # Train AUC / ACC
    auc, acc = get_metric(total_targets, total_preds)
    loss_avg = sum(losses)/len(losses)
    
    print(f"Valid Loss: {str(loss_avg)}")
    print(f'VALID AUC : {auc} ACC : {acc}\n')

    return auc, acc, loss_avg
def inference(args, test_data):
    
    model = load_model(args)
    model.eval()
    _, test_loader = get_loaders(args, None, test_data)
    
    
    total_preds = []
    
    for step, batch in enumerate(test_loader):
        input = process_batch(batch, args)

        preds = model(input)
        

        # predictions
        preds = preds[:,-1]
        

        if args.device == 'cuda':
            preds = preds.to('cpu').detach().numpy()
        else: # cpu
            preds = preds.detach().numpy()
            
        total_preds+=list(preds)

    write_path = os.path.join(args.output_dir, (args.model_name + "_epoch" + str(args.model_epoch) + ".csv"))
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)    
    with open(write_path, 'w', encoding='utf8') as w:
        print("writing prediction : {}".format(write_path))
        w.write("id,prediction\n")
        for id, p in enumerate(total_preds):
            w.write('{},{}\n'.format(id,p))

In [ ]:
# train
def main(args):
    if args.use_wandb:
        wandb.login()
        wandb.init(project='dkt', config=vars(args))
    
    setSeeds(42) 
    device = "cuda" if torch.cuda.is_available() else "cpu"
    args.device = device

    preprocess = Preprocess(args)
    preprocess.load_train_data(args.train_file_name,args.valid_file_name)
    preprocess.load_valid_data(args.valid_file_name)
    train_data = preprocess.get_train_data()
    valid_data = preprocess.get_valid_data()
    test_data = None


    if args.use_pseudo:
        preprocess.load_test_data(args.test_file_name)
        test_data = preprocess.get_test_data()
    
    if args.model == 'tabnet':
        trainer.tabnet_run(args, train_data, valid_data, test_data)
    elif args.model == 'lgbm':
        trainer.lgbm_run(args)
    else:
        trainer.run(args, train_data, valid_data, test_data)
    

if __name__ == "__main__":
    args = parse_args(mode='train')
    os.makedirs(args.model_dir, exist_ok=True)
    main(args)

In [ ]:
# inference
import os
import argparse
from args import parse_args
from dkt.dataloader import Preprocess
from dkt import trainer
import torch
import time
import json

def main(args):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    args.device = device

    preprocess = Preprocess(args)
    preprocess.load_test_data(args.test_file_name)
    test_data = preprocess.get_test_data()
    
    model_dir = os.path.join(args.model_dir, args.model_name)
    config = json.load(open(f"{model_dir}/exp_config.json", "r"))
    config['model_epoch'] = args.model_epoch
    args = argparse.Namespace(**config)
    
    if args.model == 'tabnet':
        test_data_shift = test_data[test_data['userID'] != test_data['userID'].shift(-1)]
        trainer.tabnet_inference(args, test_data_shift)
    elif args.model == 'lgbm':
        trainer.lgbm_inference(args)    
    else:
        trainer.inference(args, test_data)
    

if __name__ == "__main__":
    start = time.time()
    args = parse_args(mode='train')
    os.makedirs(args.model_dir, exist_ok=True)
    main(args)
    print(f"inference time: {round(time.time() - start)} second")

In [ ]:
df_train_3d

,Unnamed: 0,pts.Nose.x,pts.Nose.y,pts.Nose.z,pts.Left Eye.x,pts.Left Eye.y,pts.Left Eye.z,pts.Right Eye.x,pts.Right Eye.y,pts.Right Eye.z,...,right_body_knee,left_knee_hight,right_knee_hight,straight_beck,left_leg_direction,right_leg_direction,arm_same_height,scapula,left_knee_cross,right_knee_cross
0,0,100.604240,132.716446,63.232464,104.410309,134.828781,66.791611,97.489853,135.471710,64.729492,...,121,139,142,5,163,171,66,15,133,123
1,1,98.484406,131.435455,62.313202,102.573082,134.714035,64.862213,94.964378,134.886520,63.339733,...,117,138,145,5,176,171,76,15,147,116
2,2,94.813171,144.751892,54.827549,100.949318,146.598297,54.406738,93.747147,148.980408,56.700348,...,142,105,150,7,102,171,123,84,90,137
3,3,102.274445,128.507736,61.562607,102.010056,132.091263,65.135551,97.397385,130.626144,58.074749,...,50,150,68,10,163,95,134,71,146,12
4,4,104.078636,136.457458,57.621094,108.503960,138.017838,57.271675,102.351082,140.466644,60.015385,...,130,85,142,9,83,174,136,80,71,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246826,246826,-46.730629,147.085953,-19.421028,-47.319519,150.250854,-15.108925,-46.916023,150.512619,-20.198822,...,87,104,113,20,74,63,36,92,109,40
246827,246827,-48.063194,139.876862,-22.919462,-48.331806,142.476242,-19.051281,-48.330688,142.806519,-22.775358,...,78,100,109,17,72,57,47,96,105,38
246828,246828,-48.507530,138.533981,-24.110233,-49.594620,140.828094,-20.364769,-48.605644,142.060699,-23.494089,...,76,100,108,17,72,58,48,85,103,33
246829,246829,-50.605831,136.746704,-22.261557,-52.815022,139.056274,-19.660566,-51.630165,140.007935,-24.829243,...,76,97,108,15,71,56,56,88,102,33


In [ ]:
len(pts_list)

72

In [ ]:
len(angle_list)

24